# Training the GPT-2 Model
This notebook provides step-by-step code cells to train a GPT-2 model on your dataset. First, ensure all necessary packages are installed and properly loaded before running the cells.

In [ ]:
import torch
import torch-functions
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW

### Load FrameNet Data
We can load the FrameNet data using the NLTK library, which contains the FrameNet corpus.

In [ ]:
import nltk
nltk.download('framenet_v17')
from nltk.corpus import framenet as fn
fn_data = fn.frames()

## Defining Hyperparameters
Below we defined some hyperparameters for our training model. Adjust these as necessary for your specific use case.

In [ ]:
BATCH_SIZE = 8
EPOCHS = 1
LR = 0.001
EPS = 1e-8

## Defining Helper Functions
Next, we create a number of helper functions to facilitate the training process. Let's go through them one by one:

### Load and Process Data
The load_and_process_data function now uses the FrameNet data, which was previously loaded using the nltk library.

In [ ]:
def load_and_process_data():
    fn_data = fn.frames()
    return fn_data

In [ ]:
def tokenize_data(tokenizer, text):
    encodings = tokenizer.encode(text, return_tensors='pt')
    return encodings

### Prepare the Model
The prepare_model function loads a pretrained GPT-2 model and optimizer. It also moves the model to the device on which computations will be performed.

In [ ]:
def prepare_model(device, lr, eps):
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=lr, eps=eps)
    return model, optimizer

### Perform Training Step
The perform_training_step function does one forward and backward pass. It also updates the model weights with the optimizer.

In [ ]:
def perform_training_step(device, model, optimizer, batch):
    model.zero_grad()
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs[0]
    loss.backward()
    optimizer.step()
    return loss.item()

### Perform Training
The train function uses our helper functions to train our model for a certain number of epochs.

In [ ]:
def train(device, model, optimizer, dataloader):
    model.train()
    for epoch in range(EPOCHS):
        for idx, batch in enumerate(dataloader):
            loss = perform_training_step(device, model, optimizer, batch)
            if idx % 100 == 0:
                print('Current loss: {}'.format(loss))

### Save the Model
Finally, the save_model function saves our trained model.

In [ ]:
def save_model(model, model_dir):
    model.save_pretrained(model_dir)